# Import Libraries

In [1]:
import torch
def format_pytorch_version(version):
  return version.split('+')[0]
TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
def format_cuda_version(version):
  return 'cu' + version.replace('.', '')
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install pyg
import os.path as osp
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 2.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 3.5 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.3 MB 2.7 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 747 kB 2.6 MB/s 
     |████████████████████████████████| 370 kB 5.1 MB/s 
     |████████████████████████████████| 482 kB 44.2 MB/s 
     |████████████████████████████████| 41 kB 501 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=0e6e3c73e7836c30c0e6a712d7367a6212a0c276abe99514470b0d1205997f6f
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built to

In [2]:
#@title 1.Classification task on Cora Dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [3]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.0005)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3143, Val: 0.2380, Test: 0.2340
Epoch: 002, Train: 0.2214, Val: 0.1940, Test: 0.1830
Epoch: 003, Train: 0.3357, Val: 0.2440, Test: 0.2440
Epoch: 004, Train: 0.6429, Val: 0.5120, Test: 0.4710
Epoch: 005, Train: 0.8143, Val: 0.6640, Test: 0.6510
Epoch: 006, Train: 0.8929, Val: 0.7260, Test: 0.7100
Epoch: 007, Train: 0.8857, Val: 0.7260, Test: 0.7180
Epoch: 008, Train: 0.9071, Val: 0.7360, Test: 0.7460
Epoch: 009, Train: 0.9214, Val: 0.7420, Test: 0.7740
Epoch: 010, Train: 0.9357, Val: 0.7460, Test: 0.7700
Epoch: 011, Train: 0.9571, Val: 0.7500, Test: 0.7780
Epoch: 012, Train: 0.9571, Val: 0.7660, Test: 0.7890
Epoch: 013, Train: 0.9500, Val: 0.7720, Test: 0.8010
Epoch: 014, Train: 0.9357, Val: 0.7980, Test: 0.8090
Epoch: 015, Train: 0.9500, Val: 0.7880, Test: 0.8100
Epoch: 016, Train: 0.9429, Val: 0.7900, Test: 0.8080
Epoch: 017, Train: 0.9286, Val: 0.7880, Test: 0.7970
Epoch: 018, Train: 0.9429, Val: 0.7900, Test: 0.7970
Epoch: 019, Train: 0.9357, Val: 0.7880, Test: 

In [4]:
#@title 2.Classification task on Citeseer Dataset
dataset = 'Citeseer'
dataset = Planetoid(root='/tmp/Citeseer', name=dataset, transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [5]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.0005)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2417, Val: 0.1740, Test: 0.1990
Epoch: 002, Train: 0.2083, Val: 0.1500, Test: 0.1670
Epoch: 003, Train: 0.2083, Val: 0.1540, Test: 0.1780
Epoch: 004, Train: 0.2583, Val: 0.1840, Test: 0.1920
Epoch: 005, Train: 0.3083, Val: 0.2060, Test: 0.2200
Epoch: 006, Train: 0.5333, Val: 0.2360, Test: 0.2520
Epoch: 007, Train: 0.6167, Val: 0.3380, Test: 0.3430
Epoch: 008, Train: 0.6750, Val: 0.4340, Test: 0.4140
Epoch: 009, Train: 0.6833, Val: 0.4480, Test: 0.4300
Epoch: 010, Train: 0.7000, Val: 0.4600, Test: 0.4270
Epoch: 011, Train: 0.7417, Val: 0.4620, Test: 0.4300
Epoch: 012, Train: 0.7833, Val: 0.4620, Test: 0.4510
Epoch: 013, Train: 0.8500, Val: 0.5060, Test: 0.4840
Epoch: 014, Train: 0.8917, Val: 0.5960, Test: 0.5650
Epoch: 015, Train: 0.9167, Val: 0.6500, Test: 0.6610
Epoch: 016, Train: 0.9083, Val: 0.6900, Test: 0.6910
Epoch: 017, Train: 0.9083, Val: 0.6860, Test: 0.7030
Epoch: 018, Train: 0.9000, Val: 0.6820, Test: 0.6790
Epoch: 019, Train: 0.9083, Val: 0.6620, Test: 

In [6]:
#@title 3.Classification task on Pubmed Dataset
dataset = 'Pubmed'
dataset = Planetoid(root='/tmp/Pubmed', name = dataset, transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [7]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads=8, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3833, Val: 0.4000, Test: 0.4240
Epoch: 002, Train: 0.7167, Val: 0.5660, Test: 0.5620
Epoch: 003, Train: 0.6500, Val: 0.5300, Test: 0.5190
Epoch: 004, Train: 0.6333, Val: 0.5340, Test: 0.5210
Epoch: 005, Train: 0.6333, Val: 0.5440, Test: 0.5300
Epoch: 006, Train: 0.8167, Val: 0.5660, Test: 0.5520
Epoch: 007, Train: 0.8667, Val: 0.6680, Test: 0.6440
Epoch: 008, Train: 0.8333, Val: 0.7100, Test: 0.6730
Epoch: 009, Train: 0.8333, Val: 0.7120, Test: 0.6930
Epoch: 010, Train: 0.8333, Val: 0.7180, Test: 0.7010
Epoch: 011, Train: 0.8500, Val: 0.7280, Test: 0.7040
Epoch: 012, Train: 0.8667, Val: 0.7340, Test: 0.7070
Epoch: 013, Train: 0.9000, Val: 0.7380, Test: 0.7060
Epoch: 014, Train: 0.9167, Val: 0.7280, Test: 0.6990
Epoch: 015, Train: 0.9167, Val: 0.7320, Test: 0.7070
Epoch: 016, Train: 0.9167, Val: 0.7280, Test: 0.7110
Epoch: 017, Train: 0.9167, Val: 0.7380, Test: 0.7030
Epoch: 018, Train: 0.9000, Val: 0.7320, Test: 0.6980
Epoch: 019, Train: 0.8833, Val: 0.7260, Test: 